In [5]:
import numpy as np
import nltk
import json
import matplotlib.pyplot as plt
from autocorrect import Speller
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import collections
from functools import reduce
from fuzzywuzzy import fuzz, process
import pymorphy2
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import wordnet

c:\Users\fktrc\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [7]:
path_to_js = input()
custombannedwords = ['!', ',', '.', '№', '-']
similoef = 1

In [1]:
#кастом датасет для удобной работы
class Customdataset:
    def __init__(self, path : str(), stop_words=None, lang='ru'):
        with open(path_to_js) as fl:
            self.dataset = json.loads(fl.read())
        assert('ru' == lang)
        
        if not(stop_words):
            self.banned = stopwords.words('russian')
        else:
            self.banned = stop_words
            
        self.allnonfil = [text['text'] for text in self.dataset]
        self.tknzd = [word_tokenize(text['text'], 'russian') for text in self.dataset]
        #filter
        self.tknzd = list(map(lambda x: [word.lower() for word in x if word not in self.banned], self.tknzd))
        #build vocab
        self.vocab = set(reduce(lambda x, y: x + y, self.tknzd))
    

    #получить i предложение с отделенными токенами
    def __getitem__(self, ind):
        assert(ind < len(self.tknzd))
        return self.tknzd[ind]
    
    #нормализация токенов
    def norm(self):
        mor = pymorphy2.MorphAnalyzer()
        for i in range(len(self.tknzd)):
            for indword in range(len(self.tknzd[i])):
                self.tknzd[i][indword] = mor.parse(self.tknzd[i][indword])[0].normal_form
        return self.tknzd


    #получить датасет со стоп словами (list)
    def getNfil(self):
        return (self.allnonfil, self.banned)

    #получить словарь слов
    def getVocab(self):
        return self.vocab
    
    #длина датасета
    def __len__(self):
        return len(self.tknzd)


    #равенство исходя из разности множеств уник. слов
    def eq(self, ind1, ind2):
        if(len(set(self[ind1]) - set(self[ind2])) <= 1):
            return True
        return False
    
    #схожесть по метрике Джекарда
    def eqJack(self, ind1, ind2):
        intersec = set(self.tknzd[ind1]) & set(self.tknzd[ind2])
        fset = set(self.tknzd[ind1])
        sset = set(self.tknzd[ind2])
        return len(intersec) / len((fset | sset - intersec))
    
    #метод получения местоимений, имен собственных и глаголов
    def getRelation(self, tokens):
        mor = pymorphy2.MorphAnalyzer()
        ims, verbs = [], []
        for indword in range(len(tokens)):
            word = tokens[indword]
            if mor.parse(word)[0].tag.POS in ['VERB', 'INFN']:
                if tokens[indword - 1] == 'не':
                    verbs.append('не '+mor.parse(word)[0].normal_form)
                else:
                    verbs.append(mor.parse(word)[0].normal_form)
            if word in names or mor.parse(word)[0].tag.POS in ['NPRO']:
                ims.append(word)
        return [ims, verbs]

    #метод схожести двух предложений, исходя из полученных местоимений, глаголов, имен собственных
    def eqs(self, senF, senS):
        tksF = self.getRelation(senF)
        tksS = self.getRelation(senS)
        if len(tksF[0]) != len(tksS[0]) or len(tksS[1]) != len(tksF[1]):
            return False
        if len(tksF[0]) == len(tksF[1]) == len(tksS[0]) == len(tksS[1]) == 0:
            return True

        for i in range(len(tksF[0])):
            if tksF[0][i] != tksS[0][i]:
                return False
        
        for i in range(len(tksF[1])):
            if tksF[1][i] != tksS[1][i]:
                return False

        return True


In [2]:
#датасет без нормализации
cst = Customdataset(path_to_js, stop_words=custombannedwords)
#датасет с нормализованными токенами
nrmzd = Customdataset(path_to_js, stop_words=custombannedwords).norm()

NameError: name 'path_to_js' is not defined

In [ ]:
#функция получения схожести по косинусовой мере
def getCos(Fsen, Ssen):
    return np.dot(Fsen, Ssen.T) / (np.linalg.norm(Fsen) * np.linalg.norm(Ssen))

In [3]:
#получение матрицы tf-idf
model = TfidfVectorizer()
mat = model.fit_transform(cst.getNfil()[0]).toarray()

#получение словаря -  i: [дубликаты]
res = {}


levin_thres = 89
jack_thres = 0.6
cos_thres = 0.559

sim = []

texts = cst.getNfil()[0]

for i in range(len(cst)):
    for j in range(i+1, len(cst)):
        k = fuzz.token_set_ratio(nrmzd[i], nrmzd[j])
        '''
        k >= 89: расстояние левенштейна
        cst.eqs(nrmzd[i], nrmzd[j]) принцип сравнения набора местоимений и глаголов
        cst.eqJack(i, j) >= 0.6: сравнение по коэфф. Джакарда 
        and getCos(mat[i], mat[j]) > 0.559: сравнение по коэф. кос. меры
        '''
        if k >= levin_thres and cst.eqs(nrmzd[i], nrmzd[j]) and cst.eqJack(i, j) >= jack_thres and getCos(mat[i], mat[j]) > cos_thres:
            sim.append((i, j))
            if i in res:
                
                res[texts[i]].append(texts[j])
            else:
                res[texts[i]] = [texts[j]]

NameError: name 'TfidfVectorizer' is not defined

In [4]:
def extract(s: str, arr: list[str], sim: list[(int, int)]):
    res = []
    for cop in sim:
        if s == texts[cop[0]]:
            res.append(texts[cop[1]])
    res = sorted(res, reverse=True)
    return res
result = {}
for i in range(len(texts)):
    result[texts[i]] = extract(texts[i], texts[:i] + texts[i + 1:], sim)

with open('result2.json', 'w') as file:
    json.dump(result, file, indent=4, ensure_ascii=False)

NameError: name 'texts' is not defined

In [ ]:
res

{2: [(['никто', 'больше', 'туда', 'не', 'ходит'], 1.0000000000000002)],
 3: [(['у', 'него', 'тогда', 'не', 'было', 'с', 'собой', 'денег'],
   1.0000000000000002)],
 6: [(['я', 'не', 'хочу', 'с', 'тобой', 'больше', 'играть'],
   1.0000000000000002)],
 9: [(['том', 'хочет', 'меня', 'сейчас', 'видеть', '?'], 1.0)],
 13: [(['я', 'больше', 'его', 'не', 'увижу'], 1.0)],
 16: [(['том', 'и', 'мэри', 'объявили', 'сегодня', 'о', 'своей', 'помолвке'],
   0.7333109034970221)],
 18: [(['я', 'больше', 'не', 'могу', 'ждать'], 1.0)],
 19: [(['мост', 'очень', 'длинный', 'и', 'очень', 'высокий'],
   0.9530029237098314)],
 21: [(['пусть', 'это', 'свиньи', 'едят'], 1.0)],
 23: [(['ты', 'хотел', 'рассказать', 'мне', 'о', 'свободе', '?'],
   1.0000000000000002)],
 24: [(['что', 'джон', 'пел', 'на', 'сцене', '?'], 1.0)],
 25: [(['я', 'написал', 'вчера', 'письмо'], 0.6985203176230008)],
 29: [(['они', 'только', 'что', 'их', 'нашли'], 1.0)],
 31: [(['том', 'ничего', 'не', 'знает', 'о', 'семье', 'мэри'],
   1.0

In [ ]:
for item in res:
    print(cst[item])
    print(res[item])
    print()
    

['никто', 'туда', 'больше', 'не', 'ходит']
[(['никто', 'больше', 'туда', 'не', 'ходит'], 1.0000000000000002)]

['у', 'него', 'с', 'собой', 'не', 'было', 'тогда', 'денег']
[(['у', 'него', 'тогда', 'не', 'было', 'с', 'собой', 'денег'], 1.0000000000000002)]

['я', 'больше', 'не', 'хочу', 'с', 'тобой', 'играть']
[(['я', 'не', 'хочу', 'с', 'тобой', 'больше', 'играть'], 1.0000000000000002)]

['том', 'меня', 'сейчас', 'хочет', 'видеть', '?']
[(['том', 'хочет', 'меня', 'сейчас', 'видеть', '?'], 1.0)]

['я', 'его', 'больше', 'не', 'увижу']
[(['я', 'больше', 'его', 'не', 'увижу'], 1.0)]

['том', 'и', 'мэри', 'объявили', 'сегодня', 'о', 'своей', 'пбмолвке']
[(['том', 'и', 'мэри', 'объявили', 'сегодня', 'о', 'своей', 'помолвке'], 0.7333109034970221)]

['я', 'не', 'могу', 'больше', 'ждать']
[(['я', 'больше', 'не', 'могу', 'ждать'], 1.0)]

['мост', 'очень', 'длинный', 'и', 'высокий']
[(['мост', 'очень', 'длинный', 'и', 'очень', 'высокий'], 0.9530029237098314)]

['пусть', 'свиньи', 'это', 'едят']
[([

In [6]:
len(res)

NameError: name 'res' is not defined

In [7]:
res = {}
isfalse = []
for i in range(len(cst)):
    flg = 0
    for j in range(i+1, len(cst)):
        if 0.7 > cst.eqJack(i, j) > 0.5:
            if i in res:
                res[i].append((cst[j], cst.eqJack(i, j)))
            else:
                res[i] = [(cst[j], cst.eqJack(i, j))]
            flg = 1
    if flg == 0:
        isfalse.append(cst[i])

NameError: name 'cst' is not defined

In [8]:
for item in res:
    print(cst[item])
    print(res[item])
    print()
    

In [9]:
a = ['какая', 'гора', 'самая', 'высокая', 'в', 'мире', '?']
b = ['какая', 'гора', 'самая', 'высокая', 'в', 'европе', '?']

In [10]:
model = TfidfVectorizer()
mat = model.fit_transform(cst.getNfil()[0]).toarray()

NameError: name 'cst' is not defined

In [130]:
def getCos(Fsen, Ssen):
    return np.dot(Fsen, Ssen.T) / (np.linalg.norm(Fsen) * np.linalg.norm(Ssen))

In [131]:
getCos(mat[116], mat[83])

0.5591057621002352

In [134]:
mat[149]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [149]:
res = {}
isfalse = []
for i in range(len(cst)):
    flg = 0
    for j in range(i+1, len(cst)):
        if 0.66 > getCos(mat[i], mat[j]) > 0.4:
            if i in res:
                res[i].append((cst[j], getCos(mat[i], mat[j])))
            else:
                res[i] = [(cst[j], getCos(mat[i], mat[j]))]
            flg = 1
    if flg == 0:
        isfalse.append(cst[i])

In [150]:
for item in res:
    print(cst[item])
    print(res[item])
    print()
    

['почему', 'она', 'так', 'со', 'мной', 'поступает', '?']
[(['почему', 'она', 'так', 'с', 'ней', 'поступает', '?'], 0.6412369613308364)]

['никто', 'туда', 'больше', 'не', 'ходит']
[(['том', 'никогда', 'туда', 'больше', 'не', 'вернётся'], 0.45071225348547567)]

['тому', 'было', 'тогда', 'всего', 'тринадцать', 'лет']
[(['сколько', 'тебе', 'тогда', 'было', 'лет', '?'], 0.4809014706540922), (['сколько', 'шебе', 'тогда', 'было', 'лет', '?'], 0.44807992612865755)]

['что', 'сделал', 'том', 'с', 'деньгами', '?']
[(['что', 'сделёл', 'том', 'с', 'деньгами', '?'], 0.6398923908428648), (['я', 'только', 'что', 'это', 'сделал'], 0.43018414080689305), (['что', 'ты', 'ещё', 'сделал', '?'], 0.4266971961472162)]

['том', 'меня', 'сейчас', 'хочет', 'видеть', '?']
[(['том', 'не', 'хочет', 'никого', 'видеть'], 0.6524458415621264), (['том', 'не', 'хочет', 'сегодня', 'никого', 'видеть'], 0.5971697648565751), (['том', 'сегодня', 'не', 'хочет', 'никого', 'видеть'], 0.5971697648565751)]

['тебе', 'это', 'всё',

{}